# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings(action='once')
import pickle
from imblearn.over_sampling import SMOTE

In [22]:
import os
# os.chdir('PRS-PM-ISY5002-GROUP5/SystemCode')
print(os.getcwd())

/home/jupyter/temp25Oct-Mok-cfc9083c3125ca7ce3bf62e5/PRS-PM-ISY5002-GROUP5/SystemCode


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
%load_ext autoreload
%autoreload 2
from datapipeline import Datapipeline
dpl = Datapipeline()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Read data from file

In [24]:
file_dir = './Data Exploration/data/uc1/'
dict_X_train_file_paths = {
#    'GHL' : [file_dir + 'GHL_new_train_X_uc1.pkl'],
    'MEH' : [file_dir + 'MEH_new_train_X_uc1.pkl'],
#    'PEH' : [file_dir + 'PEH_new_train_X_uc1.pkl'],
#    'PNH' : [file_dir + 'PNH_new_train_X_uc1.pkl']
}
dict_y_train_file_paths = {
#     'GHL' : file_dir + 'GHL_data_y_train.pkl',
    'MEH' : file_dir + 'MEH_data_y_train.pkl',
#     'PEH' : file_dir + 'PEH_data_y_train.pkl',
#     'PNH' : file_dir + 'PNH_data_y_train.pkl'
}
dict_X_test_file_paths = {
#     'GHL' : [file_dir + 'GHL_new_test_X_uc1.pkl'],
    'MEH' : [file_dir + 'MEH_new_test_X_uc1.pkl'],
#     'PEH' : [file_dir + 'PEH_new_test_X_uc1.pkl'],
#     'PNH' : [file_dir + 'PNH_new_test_X_uc1.pkl']
}
dict_y_test_file_paths = {
#     'GHL' : file_dir + 'GHL_data_y_test.pkl',
    'MEH' : file_dir + 'MEH_data_y_test.pkl',
#     'PEH' : file_dir + 'PEH_data_y_test.pkl',
#     'PNH' : file_dir + 'PNH_data_y_test.pkl'
}

In [25]:
dict_df_X_train = {}
dict_df_y_train = {}
dict_df_X_test = {}
dict_df_y_test = {}

for hosp in dict_X_train_file_paths:
    dict_df_X_train[hosp] = pd.concat([pd.read_pickle(file_path)
                                       for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {dict_df_X_train[hosp].shape}')
    
    dict_df_y_train[hosp] = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {dict_df_y_train[hosp].shape}')

    dict_df_X_test[hosp] = pd.concat([pd.read_pickle(file_path)
                                      for file_path in dict_X_test_file_paths[hosp]])
    print(f'X_test {hosp} {dict_df_X_test[hosp].shape}')

    dict_df_y_test[hosp] = pd.read_pickle(dict_y_test_file_paths[hosp])
    print(f'y_test {hosp} {dict_df_y_test[hosp].shape}')

X_train MEH (98296, 4975)
y_train MEH (98296,)
X_test MEH (32766, 4975)
y_test MEH (32766,)


In [26]:
def convert_y(df):
    df1 = dpl.bin_column(df.to_frame(), col='WRITE_OFF', bin_thresh = [500])
    df1 = pd.get_dummies(df1['bin_WRITE_OFF'])
    df1 = df1.drop(0, axis=1)
    df1.columns = ['WRITE_OFF_LABEL']
    return df1

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
for hosp in dict_df_y_train:
    dict_df_y_train[hosp] = convert_y(dict_df_y_train[hosp])
    dict_df_y_test[hosp] = convert_y(dict_df_y_test[hosp])

## DECISION TREE

### Decision Tree WITHOUT SMOTE

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.tree import export_graphviz
from subprocess import call

In [29]:
criterion = 'entropy'
rand_seed = 0

In [30]:
for hosp in dict_df_X_train:
    model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"Accuracy on {hosp} training set without SMOTE decision tree: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"Accuracy on {hosp} test set without SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_dt_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'decision tree\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'decision tree\n', classification_report(dict_df_y_test[hosp], y_pred)) 

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on MEH training set without SMOTE decision tree: 1.0
Accuracy on MEH test set without SMOTE decision tree: 0.9973753280839895


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_no_smote_dt_model.pkl'>
  if __name__ == '__main__':


MEH decision tree
 [[32674    44]
 [   42     6]]
MEH decision tree
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.12      0.12      0.12        48

    accuracy                           1.00     32766
   macro avg       0.56      0.56      0.56     32766
weighted avg       1.00      1.00      1.00     32766



### Decision Tree WITH SMOTE

#### Decision Tree WITH SMOTE SS = 0.1 ~ 1.0

In [31]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
        model.fit(X_train_ss, y_train_ss)
        print(model)
        print(f"Accuracy on {hosp} training set with {sample} SMOTE decision tree: {model.score(X_train_ss, y_train_ss)}")
        print(f"Accuracy on {hosp} test set with {sample} SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_dt_model.pkl'
        pickle.dump(model, open(filename, 'wb'))
        
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'decision tree \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'decision tree \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on MEH training set with 0.1 SMOTE decision tree: 1.0
Accuracy on MEH test set with 0.1 SMOTE decision tree: 0.9971311725569187


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.1_smote_dt_model.pkl'>
  del sys.path[0]


MEH decision tree 
 [[32666    52]
 [   42     6]]
MEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.10      0.12      0.11        48

    accuracy                           1.00     32766
   macro avg       0.55      0.56      0.56     32766
weighted avg       1.00      1.00      1.00     32766

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on MEH training set with 0.2 SMOTE decision tree: 1.0
Accuracy on MEH test set with 0.2 SMOTE decision tree: 0.9976805224928279


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.2_smote_dt_model.pkl'>
  del sys.path[0]


...

MEH decision tree 
 [[32665    53]
 [   41     7]]
MEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.12      0.15      0.13        48

    accuracy                           1.00     32766
   macro avg       0.56      0.57      0.56     32766
weighted avg       1.00      1.00      1.00     32766

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on MEH training set with 0.9 SMOTE decision tree: 1.0
Accuracy on MEH test set with 0.9 SMOTE decision tree: 0.9971616919978026


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.9_smote_dt_model.pkl'>
  del sys.path[0]


MEH decision tree 
 [[32666    52]
 [   41     7]]
MEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.12      0.15      0.13        48

    accuracy                           1.00     32766
   macro avg       0.56      0.57      0.56     32766
weighted avg       1.00      1.00      1.00     32766

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on MEH training set with 1.0 SMOTE decision tree: 1.0
Accuracy on MEH test set with 1.0 SMOTE decision tree: 0.9970396142342672


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_1.0_smote_dt_model.pkl'>
  del sys.path[0]


MEH decision tree 
 [[32663    55]
 [   42     6]]
MEH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.10      0.12      0.11        48

    accuracy                           1.00     32766
   macro avg       0.55      0.56      0.55     32766
weighted avg       1.00      1.00      1.00     32766



## LOGISTIC REGRESSION

### Logistic Regression Without SMOTE

In [32]:
from sklearn.linear_model import LogisticRegression

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
reg_strength = 0.01

In [34]:
for hosp in dict_df_X_train:
    model = LogisticRegression(C=reg_strength).fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"{hosp} Training set without SMOTE log reg score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_logreg_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
MEH Training set without SMOTE log reg score: 0.9983315699519818
MEH Test set without SMOTE log reg score: 0.9983824696331564


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_no_smote_logreg_model.pkl'>
  


MEH log reg 
 [[32712     6]
 [   47     1]]
MEH log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.14      0.02      0.04        48

    accuracy                           1.00     32766
   macro avg       0.57      0.51      0.52     32766
weighted avg       1.00      1.00      1.00     32766



### Logistic Regression With SMOTE

#### Logistic Regression With SMOTE ss=0.1 ~ 1.0

In [35]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = LogisticRegression(C=reg_strength).fit(X_train_ss, y_train_ss)
        print(model)
        print(f"{hosp} Training set with {sample} SMOTE log reg score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        filename = file_dir + f'{hosp}_uc1_{sample}_smote_logreg_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    htt

LogisticRegression(C=0.01)
MEH Training set with 0.1 SMOTE log reg score: 0.9596387216303844
MEH Test set with 0.1 SMOTE log reg score: 0.9923396203381554


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.1_smote_logreg_model.pkl'>
  if sys.path[0] == '':


MEH log reg 
 [[32494   224]
 [   27    21]]
MEH log reg 
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     32718
           1       0.09      0.44      0.14        48

    accuracy                           0.99     32766
   macro avg       0.54      0.72      0.57     32766
weighted avg       1.00      0.99      0.99     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

MEH log reg 
 [[31686  1032]
 [   19    29]]
MEH log reg 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     32718
           1       0.03      0.60      0.05        48

    accuracy                           0.97     32766
   macro avg       0.51      0.79      0.52     32766
weighted avg       1.00      0.97      0.98     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=0.01)
MEH Training set with 1.0 SMOTE log reg score: 0.9821932604420351
MEH Test set with 1.0 SMOTE log reg score: 0.9664896539095403


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_1.0_smote_logreg_model.pkl'>
  if sys.path[0] == '':


MEH log reg 
 [[31638  1080]
 [   18    30]]
MEH log reg 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     32718
           1       0.03      0.62      0.05        48

    accuracy                           0.97     32766
   macro avg       0.51      0.80      0.52     32766
weighted avg       1.00      0.97      0.98     32766



## NAIVE BAYES

### Naive Bayes Without SMOTE

In [36]:
from sklearn.naive_bayes import GaussianNB

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
for hosp in dict_df_X_train:
    # Initiating the Gaussian Classifier
    model = GaussianNB()

    # Training your model 
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    filename = file_dir + f'{hosp}_uc1_no_smote_nb_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    # Score
    print(f"{hosp} Training set without SMOTE Naive Bayes score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    # Confusion Matrix
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'Naive Bayes \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'Naive Bayes \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_no_smote_nb_model.pkl'>
  if __name__ == '__main__':


MEH Training set without SMOTE Naive Bayes score: 0.9666313990396354
MEH Test set without SMOTE Naive Bayes score: 0.9658182262100958
MEH Naive Bayes 
 [[31638  1080]
 [   40     8]]
MEH Naive Bayes 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     32718
           1       0.01      0.17      0.01        48

    accuracy                           0.97     32766
   macro avg       0.50      0.57      0.50     32766
weighted avg       1.00      0.97      0.98     32766



### Naive Bayes With SMOTE

#### Naive Bayes Without SMOTE ss=0.1 ~ 1.0

In [38]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Initiating the Gaussian Classifier
        model = GaussianNB()
        
        # Training your model 
        model.fit(X_train_ss, y_train_ss)
        print(model)
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_nb_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        # Score
        print(f"{hosp} Training set with {sample} SMOTE Naive Bayes score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        # Confusion Matrix
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'Naive Bayes\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'Naive Bayes\n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.1_smote_nb_model.pkl'>
  del sys.path[0]


MEH Training set with 0.1 SMOTE Naive Bayes score: 0.9696155627605373
MEH Test set with 0.1 SMOTE Naive Bayes score: 0.9658182262100958
MEH Naive Bayes
 [[31638  1080]
 [   40     8]]
MEH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     32718
           1       0.01      0.17      0.01        48

    accuracy                           0.97     32766
   macro avg       0.50      0.57      0.50     32766
weighted avg       1.00      0.97      0.98     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

MEH Training set with 0.9 SMOTE Naive Bayes score: 0.9824090957846187
MEH Test set with 0.9 SMOTE Naive Bayes score: 0.9658182262100958
MEH Naive Bayes
 [[31638  1080]
 [   40     8]]
MEH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     32718
           1       0.01      0.17      0.01        48

    accuracy                           0.97     32766
   macro avg       0.50      0.57      0.50     32766
weighted avg       1.00      0.97      0.98     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_1.0_smote_nb_model.pkl'>
  del sys.path[0]


MEH Training set with 1.0 SMOTE Naive Bayes score: 0.9832886678826538
MEH Test set with 1.0 SMOTE Naive Bayes score: 0.9658182262100958
MEH Naive Bayes
 [[31638  1080]
 [   40     8]]
MEH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     32718
           1       0.01      0.17      0.01        48

    accuracy                           0.97     32766
   macro avg       0.50      0.57      0.50     32766
weighted avg       1.00      0.97      0.98     32766



## NEURAL NET MLP

### Neural Net MLP Without SMOTE, 2 Layers

In [39]:
from sklearn.neural_network import MLPClassifier  
from sklearn import metrics

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
list_hid_layers = [(128,128), (16,16,16), (32,32,32), (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net MLPClassifier without SMOTE

In [42]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
        model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
        print(model)
        filename = file_dir + f'{hosp}_uc1_no_smote_mlp_{hid_layers}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        predictions = model.predict(dict_df_X_test[hosp]) 

        # Score
        print(f"{hosp} Training set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
        print(f"{hosp} Test set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy without SMOTE mlp {hid_layers}: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
        print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
        print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.02101287
Iteration 2, loss = 0.00716405
Iteration 3, loss = 0.00539036
Iteration 4, loss = 0.00412206
Iteration 5, loss = 0.00320090
Iteration 6, loss = 0.00241303
Iteration 7, loss = 0.00174643
Iteration 8, loss = 0.00137889
Iteration 9, loss = 0.00119085
Iteration 10, loss = 0.00108489
Iteration 11, loss = 0.00078143
Iteration 12, loss = 0.00147814
Iteration 13, loss = 0.00072232
Iteration 14, loss = 0.00044839
Iteration 15, loss = 0.00063368
Iteration 16, loss = 0.00050779
Iteration 17, loss = 0.00065495
Iteration 18, loss = 0.00108227
Iteration 19, loss = 0.00092085
Iteration 20, loss = 0.00045970
Iteration 21, loss = 0.00038900
Iteration 22, loss = 0.00033709
Iteration 23, loss = 0.00080756
Iteration 24, loss = 0.00042463
Iteration 25, loss = 0.00028148
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_no_smote_mlp_(128, 128)_model.pkl'>
  import sys


MEH Training set without SMOTE mlp (128, 128) score: 0.9999491332302434
MEH Test set without SMOTE mlp (128, 128) score: 0.9976500030519441
MEH Accuracy without SMOTE mlp (128, 128):  0.9976500030519441
MEH mlp (128, 128)
 [[32682    36]
 [   41     7]]
MEH mlp (128, 128)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.16      0.15      0.15        48

    accuracy                           1.00     32766
   macro avg       0.58      0.57      0.58     32766
weighted avg       1.00      1.00      1.00     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

MEH Training set without SMOTE mlp (16, 16, 16, 16) score: 0.9999593065841946
MEH Test set without SMOTE mlp (16, 16, 16, 16) score: 0.9977415613745956
MEH Accuracy without SMOTE mlp (16, 16, 16, 16):  0.9977415613745956
MEH mlp (16, 16, 16, 16)
 [[32684    34]
 [   40     8]]
MEH mlp (16, 16, 16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.19      0.17      0.18        48

    accuracy                           1.00     32766
   macro avg       0.59      0.58      0.59     32766
weighted avg       1.00      1.00      1.00     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.03442338
Iteration 2, loss = 0.00762449
Iteration 3, loss = 0.00608600
Iteration 4, loss = 0.00457675
Iteration 5, loss = 0.00321350
Iteration 6, loss = 0.00231014
Iteration 7, loss = 0.00199519
Iteration 8, loss = 0.00133859
Iteration 9, loss = 0.00134106
Iteration 10, loss = 0.00093357
Iteration 11, loss = 0.00093577
Iteration 12, loss = 0.00111958
Iteration 13, loss = 0.00071130
Iteration 14, loss = 0.00066756
Iteration 15, loss = 0.00096795
Iteration 16, loss = 0.00064621
Iteration 17, loss = 0.00051670
Iteration 18, loss = 0.00099584
Iteration 19, loss = 0.00051737
Iteration 20, loss = 0.00030266
Iteration 21, loss = 0.00069540
Iteration 22, loss = 0.00055556
Iteration 23, loss = 0.00043789
Iteration 24, loss = 0.00041169
Iteration 25, loss = 0.00081335
Iteration 26, loss = 0.00027091
Iteration 27, loss = 0.00077129
Iteration 28, loss = 0.00080127
Iteration 29, loss = 0.00035613
Iteration 30, loss = 0.00030453
Iteration 31, loss = 0.00054378
Training loss did

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_no_smote_mlp_(32, 32, 32, 32)_model.pkl'>
  import sys


MEH Training set without SMOTE mlp (32, 32, 32, 32) score: 0.9998880931065354
MEH Test set without SMOTE mlp (32, 32, 32, 32) score: 0.9981077946652017
MEH Accuracy without SMOTE mlp (32, 32, 32, 32):  0.9981077946652017
MEH mlp (32, 32, 32, 32)
 [[32699    19]
 [   43     5]]
MEH mlp (32, 32, 32, 32)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.21      0.10      0.14        48

    accuracy                           1.00     32766
   macro avg       0.60      0.55      0.57     32766
weighted avg       1.00      1.00      1.00     32766



### Neural Net With SMOTE

In [43]:
list_hid_layers = [(16,16), (32,32), (64,64), (128,128), (256,256),
                   (16,16,16), (32,32,32), (64,64,64),
                   (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Neural Net Layers With SMOTE

In [ ]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        for sample in list_smote_sampling:
            sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
            X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
            model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
            model.fit(X_train_ss, y_train_ss)  
            print(model)
            filename = file_dir + f'{hosp}_uc1_{sample}_smote_mlp_{hid_layers}_model.pkl'
            pickle.dump(model, open(filename, 'wb'))
            
            predictions = model.predict(dict_df_X_test[hosp]) 

            # Score
            print(f"{hosp} Training set {sample} SMOTE mlp {hid_layers} score: {model.score(X_train_ss, y_train_ss)}")
            print(f"{hosp} Test set {sample} SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

            print(f"{hosp} Accuracy: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
            print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
            print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.08989510
Iteration 2, loss = 0.00618101
Iteration 3, loss = 0.00356185
Iteration 4, loss = 0.00205993
Iteration 5, loss = 0.00246881
Iteration 6, loss = 0.00126704
Iteration 7, loss = 0.00106518
Iteration 8, loss = 0.00091591
Iteration 9, loss = 0.00087449
Iteration 10, loss = 0.00062633
Iteration 11, loss = 0.00101437
Iteration 12, loss = 0.00084768
Iteration 13, loss = 0.00082735
Iteration 14, loss = 0.00102561
Iteration 15, loss = 0.00055801
Iteration 16, loss = 0.00085213
Iteration 17, loss = 0.00049818
Iteration 18, loss = 0.00048643
Iteration 19, loss = 0.00064849
Iteration 20, loss = 0.00043327
Iteration 21, loss = 0.00042428
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(16, 16), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.1_smote_mlp_(16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


MEH Training set 0.1 SMOTE mlp (16, 16) score: 0.9999536822603057
MEH Test set 0.1 SMOTE mlp (16, 16) score: 0.9974363669657572
MEH Accuracy:  0.9974363669657572
MEH mlp (16, 16)
 [[32676    42]
 [   42     6]]
MEH mlp (16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.12      0.12      0.12        48

    accuracy                           1.00     32766
   macro avg       0.56      0.56      0.56     32766
weighted avg       1.00      1.00      1.00     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

Iteration 1, loss = 0.07468953
Iteration 2, loss = 0.00498216
Iteration 3, loss = 0.00250482
Iteration 4, loss = 0.00158129
Iteration 5, loss = 0.00120424
Iteration 6, loss = 0.00182652
Iteration 7, loss = 0.00108851
Iteration 8, loss = 0.00095974
Iteration 9, loss = 0.00074715
Iteration 10, loss = 0.00079534
Iteration 11, loss = 0.00072073
Iteration 12, loss = 0.00091454
Iteration 13, loss = 0.00135317
Iteration 14, loss = 0.00120701
Iteration 15, loss = 0.00061669
Iteration 16, loss = 0.00041859
Iteration 17, loss = 0.00051843
Iteration 18, loss = 0.00043026
Iteration 19, loss = 0.00060013
Iteration 20, loss = 0.00058343
Iteration 21, loss = 0.00060887
Iteration 22, loss = 0.00117440
Iteration 23, loss = 0.00040469
Iteration 24, loss = 0.00132348
Iteration 25, loss = 0.00050793
Iteration 26, loss = 0.00037106
Iteration 27, loss = 0.00037696
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(16, 16, 16), max_iter

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/MEH_uc1_0.5_smote_mlp_(16, 16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


MEH Training set 0.5 SMOTE mlp (16, 16, 16) score: 0.9999660337624401
MEH Test set 0.5 SMOTE mlp (16, 16, 16) score: 0.9979246780198987
MEH Accuracy:  0.9979246780198987
MEH mlp (16, 16, 16)
 [[32694    24]
 [   44     4]]
MEH mlp (16, 16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     32718
           1       0.14      0.08      0.11        48

    accuracy                           1.00     32766
   macro avg       0.57      0.54      0.55     32766
weighted avg       1.00      1.00      1.00     32766



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.05307272
Iteration 2, loss = 0.00264749
Iteration 3, loss = 0.00120335
Iteration 4, loss = 0.00088712
Iteration 5, loss = 0.00082707


## KNN

In [ ]:
# Be careful when running this, about 2 full days are needed to run this!!!
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
for hosp in dict_df_X_train:
    test_scores = []
    train_scores = []

    for i in range(1,15):

        knn = KNeighborsClassifier(i)
        knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])

        train_scores.append(knn.score(dict_df_X_train[hosp],dict_df_y_train[hosp]))
        test_scores.append(knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp]))

    ## score that comes from testing on the same datapoints that were used for training
    max_train_score = max(train_scores)
    train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
    print(f'{hosp} Max train score {max_train_score*100} % and k = {list(map(lambda x: x+1, train_scores_ind))}')

    ## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
    max_test_score = max(test_scores)
    test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
    print(f'{hosp} Max test score {max_test_score*100} % and k = {list(map(lambda x: x+1, test_scores_ind))}')

    plt.figure(figsize=(12,5))
    plt.plot(range(1,15),train_scores,marker='*',label='Train Score')
    plt.plot(range(1,15),test_scores,marker='o',label='Test Score')
    plt.title(f'{hosp} KNN')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# From the above, the best KNN is at k=8
#Setup a knn classifier with k neighbors
best_knn = KNeighborsClassifier(8)

### KNN without SMOTE

In [ ]:
n_neighbours = 8
for hosp in dict_df_X_train:
    # Fit the KNN model with training data and score the test data
    best_knn = KNeighborsClassifier(n_neighbours)
    
    best_knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])
    best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
    
    # save the model to disk
    filename = f'{hosp}_best_knn_without_smote.pkl'
    pickle.dump(best_knn, open(filename, 'wb'))
    
    y_pred = best_knn.predict(dict_df_X_test[hosp])
    # Score for Best KNN, No SMOTE
    print(f"{hosp} Training set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    print(f"{hosp} Accuracy without SMOTE KNN({n_neighbours}): {metrics.accuracy_score(dict_df_y_test[hosp], y_pred)}")
    print(hosp, f'KNN({n_neighbours})\n', confusion_matrix(dict_df_y_test[hosp],y_pred))  
    print(hosp, f'KNN({n_neighbours})\n', classification_report(dict_df_y_test[hosp],y_pred))  

### KNN With SMOTE

#### KNN With SMOTE ss=0.1

In [ ]:
n_neighbours = 8
list_smote_sampling = [0.1, 0.5, 1.0]
for hosp in dict_df_X_train:
    for sample in list_smote_sampling:
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Fit the KNN model with training data and score the test data
        best_knn = KNeighborsClassifier(n_neighbours)    
        best_knn.fit(X_train_ss,y_train_ss)
        #best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
        
        # save the model to disk
        filename = f'{hosp}_best_knn_{sample}_smote.pkl'
        pickle.dump(best_knn, open(filename, 'wb'))
        
        y_pred = best_knn.predict(dict_df_X_test[hosp])
        # Score for Best KNN, with SMOTE
        print(f"{hosp} Training set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy {sample} SMOTE KNN({n_neighbours}): ", metrics.accuracy_score(dict_df_y_test[hosp], y_pred))
        print(confusion_matrix(dict_df_y_test[hosp],y_pred))  
        print(classification_report(dict_df_y_test[hosp],y_pred)) 